In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import networkx as nx
import subprocess
import glob
from pathlib import Path
from matplotlib import pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

In [3]:
dataset_name = 'street'
feature_type = 'sift_default'
matching_type = 'sift_default'
geometric_verification_type = 'default'
colmap_path = 'colmap'

In [4]:
os.environ['MKL_THREADING_LAYER'] = 'GNU'
output1 = subprocess.run(['python', '../scripts/match_features.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--colmap_path', colmap_path,
                         '--use_gpu'], capture_output=True)

In [5]:
# print(str(output1.stdout, 'utf-8'))
# print(str(output1.stderr, 'utf-8'))

In [6]:
score_version = '2'
min_num_valid_depths = '5'
max_num_neighbors = '80'
square_radius = '20'

In [7]:
output2 = subprocess.run(['python', '../scripts/disambiguate_cui.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--score_version', score_version,
                         '--min_num_valid_depths', min_num_valid_depths,
                         '--max_num_neighbors', max_num_neighbors,
                         '--square_radius', square_radius,
                         '--parallel',
                         '--plot'], capture_output=True)

In [8]:
# print(str(output2.stdout, 'utf-8'))
# print(str(output2.stderr, 'utf-8'))

In [9]:
root = Path.cwd().parents[0]
results_path = (root / 'results' / dataset_name /
                '_'.join([feature_type, matching_type, geometric_verification_type]))
scores_name = 'scores' + f'_cui_v{score_version}_d{min_num_valid_depths}' + \
              f'_n{max_num_neighbors}_r{square_radius}_p.npy'
scores = np.load(results_path / scores_name)

In [10]:
@interact(thres=FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01))
def plot(thres):
    valid_scores = scores.copy()
    valid_scores[scores < thres] = 0.
    graph = nx.from_numpy_matrix(valid_scores)
    pos = nx.nx_agraph.graphviz_layout(graph)
    edge_vmin = np.percentile(valid_scores[valid_scores.nonzero()], 10)
    edge_vmax = np.percentile(valid_scores[valid_scores.nonzero()], 90)
    nx.draw(graph, pos, with_labels=True, edge_cmap=plt.cm.YlOrRd,
            edge_color=[graph[u][v]['weight'] for u, v in graph.edges],
            edge_vmin=edge_vmin, edge_vmax=edge_vmax)
    plt.show()

interactive(children=(FloatSlider(value=0.1, description='thres', max=1.0, step=0.01), Output()), _dom_classes…

In [14]:
@interact(k=IntSlider(value=3, min=1, max=10, step=1))
def plot(k):
    valid = np.zeros_like(scores, dtype=np.bool)
    valid_indices = scores.argsort()[:, -k:]
    for i in range(scores.shape[0]):
        for j in valid_indices[i]:
            valid[i, j] = True
    valid_scores = np.zeros_like(scores)
    valid_scores[valid] = scores[valid] 
    graph = nx.from_numpy_matrix(valid_scores)
    pos = nx.nx_agraph.graphviz_layout(graph)
    edge_vmin = np.percentile(valid_scores[valid_scores.nonzero()], 10)
    edge_vmax = np.percentile(valid_scores[valid_scores.nonzero()], 90)
    nx.draw(graph, pos, with_labels=True, edge_cmap=plt.cm.YlOrRd,
            edge_color=[graph[u][v]['weight'] for u, v in graph.edges],
            edge_vmin=edge_vmin, edge_vmax=edge_vmax)
    plt.show()

interactive(children=(IntSlider(value=3, description='k', max=10, min=1), Output()), _dom_classes=('widget-int…

In [11]:
filter_type = 'threshold'
threshold = '0.50'
output3 = subprocess.run(['python', '../scripts/filter_matches.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--colmap_path', colmap_path,
                         '--scores_name', scores_name,
                         '--filter_type', filter_type,
                         '--threshold', threshold,
                         # '--reconstruct_unfiltered',
                         '--reconstruct_filtered'], capture_output=True)

In [13]:
filter_type = 'knn'
topk = '3'
output3 = subprocess.run(['python', '../scripts/filter_matches.py',
                         '--dataset_name', dataset_name,
                         '--feature_type', feature_type,
                         '--matching_type', matching_type,
                         '--geometric_verification_type', geometric_verification_type,
                         '--colmap_path', colmap_path,
                         '--scores_name', scores_name,
                         '--filter_type', filter_type,
                         '--topk', topk,
                         # '--reconstruct_unfiltered',
                         '--reconstruct_filtered'], capture_output=True)

In [14]:
# print(str(output3.stderr, 'utf-8'))
# print(str(output3.stdout, 'utf-8'))

In [12]:
from disambiguation.utils import visualize_model
backbone = f'_cui_v{score_version}_d{min_num_valid_depths}' + \
              f'_n{max_num_neighbors}_r{square_radius}_p'

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
input_model = results_path / ('sparse' + backbone + f'_t{threshold}') / '0'
input_format = '.bin'
visualize_model.main(input_model, input_format)

num_cameras: 1
num_images: 19
num_points3D: 3951


In [17]:
input_model = results_path / ('sparse' + backbone + f'_k{topk}') / '0'
input_format = '.bin'
visualize_model.main(input_model, input_format)

num_cameras: 1
num_images: 19
num_points3D: 3265
